<a href="https://colab.research.google.com/github/dajuanyoung/Project---Food-Desert/blob/main/DS4A_Full_Variable_List_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install libspatialindex-dev
!pip3 install rtree pygeos
!pip install geopandas
!pip install --upgrade -q pygsheets

import geopandas as gpd
import os
import json
import numpy as np
import io
import pandas as pd
from google.colab import files
import google.auth
from google.colab import auth
import pygsheets
import matplotlib.pyplot as plt
import seaborn as sns

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (563 kB/s)
Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
#documentation for pyghsheets
#https://www.pingshiuanchua.com/blog/post/intro-to-colaboratory-and-linking-it-to-google-sheets
auth.authenticate_user()
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
def import_tab(tab_name, key):
  sheet = gc.open_by_key(key)
  tab = sheet.worksheet_by_title(tab_name)
  df = tab.get_as_df()
  return df

In [ ]:
df_2011 = import_tab('Variable List', '1bMm4TrFyuRslA45cteuBiFiCsU3MLskmYPVskPsyNEI')
df_2012 = import_tab('Variable List', '1ZR087iCTz7FsNnWdc7EZ06jTgRE5FCdB7pH-WQx2Pec')
df_2014 = import_tab('Variable List', '1ZcKXu0VDKjqBSO-obBaZHSOCy1IsXpm8UfoqlUChcWA')
df_2015 = import_tab('Variable List', '1XQdbcz-OMJPvHluONqUmcuK3BG85YdDPOSnW7xKbrro')
df_2017 = import_tab('Variable List', '1jvNEwtp8vdVeqeMSurwnTr_50IHO7VjC4WUSp3vmHf0')
df_2020 = import_tab('Variable List', '14AK1FC6smT2ND_oLfn24X3JDuLOkrtT-FS22Ag4Yzgc')

NameError: ignored

In [ ]:
all_years = [df_2011,df_2012,df_2014,df_2015,df_2017,df_2020]
years = [2011,2012,2014,2015,2017,2020]

In [ ]:
def clean_df(df, year):
  df = df[['Variable Code','Variable Name']].rename(columns={'Variable Code':'code','Variable Name':'description'})
  df['dataset_year'] = year
  return df

In [ ]:
full_vars = pd.DataFrame(columns = ['code','description','dataset_year'])

for i in range(0,len(years)):
  df = clean_df(all_years[i],years[i])
  full_vars = pd.concat([full_vars,df]).reset_index(drop=True)

In [ ]:
full_vars['description'] = full_vars['description'].str.replace('*','')

In [ ]:
full_vars['var_year'] = full_vars['description'].str[-4:]

In [ ]:
full_vars = full_vars[~full_vars['var_year'].str.contains("-")]

In [ ]:
full_vars.head(2)

,code,description,dataset_year,var_year
0,HHNV1MI,"# Households no car & > 1 mi to store, 2006",2011,2006
1,PCT_HHNV1MI,"% Households no car & > 1 mi to store, 2006",2011,2006


In [ ]:
full_vars = full_vars[~full_vars['var_year'].str.contains('000 ')]
full_vars = full_vars[~full_vars['var_year'].str.contains('005 ')]
full_vars = full_vars[~full_vars['var_year'].str.contains('ions')]
full_vars = full_vars[~full_vars['var_year'].str.contains('ipts')]

In [ ]:
full_vars['var_year'] = full_vars['var_year'].astype('int')

In [ ]:
full_vars['dataset_year'] = full_vars['dataset_year'].astype('int')

In [ ]:
full_vars.groupby(['var_year']).nunique().reset_index().rename(columns={0:'num_of_vars'})

,var_year,code,description,dataset_year
0,1999,1,1,4
1,2000,12,11,4
2,2001,2,2,1
3,2002,2,5,4
4,2005,4,4,2
5,2006,29,39,4
6,2007,51,72,6
7,2008,48,50,6
8,2009,50,57,6
9,2010,50,49,6


In [ ]:
full_vars[full_vars['var_year']==2010]['code'].unique()

array(['AM_SNAPPART10', 'NSLP_PART10', 'SBP_PART10', 'SFSP_AVG_ATT10',
       'WIC_PART10', 'CACFP_AVG_ATT10', 'FMRKT10', 'SNAPS10',
       'SNAPSPTH10', 'REDEMP_SNAPS10', 'PC_SNAPBEN10', 'MILK_PRICE10',
       'SODA_PRICE10', 'MILK_SODA_PRICE10', 'PCT_NHWHITE10',
       'PCT_NHBLACK10', 'PCT_HISP10', 'PCT_NHASIAN10', 'PCT_NHNA10',
       'PCT_NHPI10', 'PCT_65OLDER10', 'PCT_18YOUNGER10', 'MEDHHINC10',
       'POVRATE10', 'CHILDPOVRATE10', 'LACCESS_POP10',
       'PCT_LACCESS_POP10', 'LACCESS_LOWI10', 'PCT_LACCESS_LOWI10',
       'LACCESS_CHILD10', 'PCT_LACCESS_CHILD10', 'LACCESS_SENIORS10',
       'PCT_LACCESS_SENIORS10', 'LACCESS_HHNV10', 'PCT_LACCESS_HHNV10',
       'SNAP_PART_RATE10', 'PCT_FREE_LUNCH10', 'PCT_REDUCED_LUNCH10',
       'SNAP_OAPP10', 'SNAP_FACEWAIVER10', 'SNAP_VEHEXCL10',
       'SNAP_BBCE10', 'SNAP_REPORTSIMPLE10', 'PCT_DIABETES_ADULTS10',
       'PCT_OBESE_ADULTS10', 'PERPOV11', 'METRO13', 'PERPOV10',
       'PERCHLDPOV10', 'POPLOSS10'], dtype=object)

In [ ]:
#2010 vars not in our full 2015 data
missing_vars = list()
for var in full_vars[full_vars['var_year']==2010]['description'].unique():
  if var not in full_vars[(full_vars['var_year']==2010) & (full_vars['dataset_year']==2015)]['description'].unique():
    #print(var in full_vars[(full_vars['var_year']==2010) & (full_vars['dataset_year']==2015)]['description'].unique())
    print(var)
    missing_vars.append(var)

Avg monthly # SNAP participants, 2010
Avg daily # School-Lunch participants, 2010
Avg daily # Schl-Breakfast participants, 2010
Avg daily # Summer-Food participants, 2010
Avg monthly # WIC participants, 2010
Avg daily served Child-&-Adult Care, 2010
# Farmers' markets, 2010
SNAP-authorized stores, 2010
SNAP-authorized stores/1,000 pop, 2010
SNAP redemptions/SNAP-authorized stores, 2010
Population-loss counties, 2010


In [ ]:
full_vars[full_vars['var_year']==2011]['description'].unique()

array(['WIC-authorized stores, 2011',
       'WIC-authorized stores/1,000 pop, 2011',
       'SNAP participants (% pop), 2011',
       'School Lunch participants (% pop), 2011',
       'School Breakfast participants (% pop), 2011',
       'Summer Food participants (% pop), 2011',
       'WIC redemptions per capita, 2011',
       'WIC redemptions/WIC-authorized stores, 2011',
       'WIC participants (% pop), 2011',
       'Child & Adult Care (% pop), 2011',
       'Soda sales tax, retail stores, 2011',
       'Soda sales tax, vending, 2011',
       'Chip & pretzel sales tax, retail stores, 2011',
       'Chip & pretzel sales tax, vending, 2011',
       'General food sales tax, retail stores, 2011',
       "Farmers' markets, 2011", "Farmers' markets/1,000 pop, 2011",
       'Grocery stores, 2011', 'Grocery stores/1,000 pop, 2011',
       'Supercenters & club stores, 2011',
       'Supercenters & club stores/1,000 pop, 2011',
       'Convenience stores, 2011', 'Convenience stores/1,000 p

In [ ]:
full_vars[full_vars['description'].isin(missing_vars)]

,code,description,dataset_year,var_year
73,AM_SNAPPART10,"Avg monthly # SNAP participants, 2010",2011,2010
85,NSLP_PART10,"Avg daily # School-Lunch participants, 2010",2011,2010
92,SBP_PART10,"Avg daily # Schl-Breakfast participants, 2010",2011,2010
95,SFSP_AVG_ATT10,"Avg daily # Summer-Food participants, 2010",2011,2010
100,WIC_PART10,"Avg monthly # WIC participants, 2010",2011,2010
106,CACFP_AVG_ATT10,"Avg daily served Child-&-Adult Care, 2010",2011,2010
140,FMRKT10,"# Farmers' markets, 2010",2011,2010
201,SNAPS10,"SNAP-authorized stores, 2010",2012,2010
204,SNAPSPTH10,"SNAP-authorized stores/1,000 pop, 2010",2012,2010
229,REDEMP_SNAPS10,"SNAP redemptions/SNAP-authorized stores, 2010",2012,2010


In [ ]:
vars11 = full_vars[full_vars['var_year']==2011]['description'].unique()
full_vars[(full_vars['description'].isin(vars11)) & (full_vars['dataset_year'] == 2020)][['description']]

,description
1076,"Grocery stores, 2011"
1079,"Grocery stores/1,000 pop, 2011"
1082,"Supercenters & club stores, 2011"
1085,"Supercenters & club stores/1,000 pop, 2011"
1088,"Convenience stores, 2011"
1091,"Convenience stores/1,000 pop, 2011"
1094,"Specialized food stores, 2011"
1097,"Specialized food stores/1,000 pop, 2011"
1106,"WIC-authorized stores, 2011"
1109,"WIC-authorized stores/1,000 pop, 2011"


In [ ]:
vars2010atlas2012 = ['SNAPS10','SNAPSPTH10','REDEMP_SNAPS10']

In [ ]:
sorted(full_vars['description'].unique())

['# Convenience stores no gas, 2007',
 '# Convenience stores no gas, 2008',
 '# Convenience stores with gas, 2007',
 '# Convenience stores with gas, 2008',
 "# Farmers' markets, 2009",
 "# Farmers' markets, 2010",
 '# Farms with direct sales, 2007',
 '# Fast-food restaurants, 2007',
 '# Fast-food restaurants, 2008',
 '# Full-service restaurants, 2007',
 '# Full-service restaurants, 2008',
 '# Grocery stores, 2007',
 '# Grocery stores, 2008',
 '# Households no car & > 1 mi to store, 2006',
 '# Households no car & > 10 mi to store, 2006',
 '# Low income & > 1 mi to store, 2006',
 '# Low income & > 10 mi to store, 2006',
 '# Recreation & fitness facilities, 2007',
 '# Recreation & fitness facilities, 2008',
 '# SNAP-authorized stores, 2008',
 '# SNAP-authorized stores, 2009',
 '# Specialized food stores, 2007',
 '# Specialized food stores, 2008',
 '# Summer-Food program sites, 2001',
 '# Supercenters and club stores, 2007',
 '# Supercenters and club stores, 2008',
 '# Vegetable acres harv